In [ ]:
import tensorflow as tf
import numpy as np
import scipy
import h5py
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.python.keras.preprocessing  import  image 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense,Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.python.keras.layers import BatchNormalization
from IPython.display import display
from tensorflow.python.keras.layers import ELU
from numpy.random import seed
from shutil import copyfile
from time import time
tf.set_random_seed(1)
seed(1)

In [ ]:
# dimensions of our images.
image_width, image_height = 32, 32
WORK_DIRECTORY="C:/tmp/dogcat_study/"
os.chdir(WORK_DIRECTORY) 
train_data_dir      = 'KaggleCatDog/train'
validation_data_dir = 'KaggleCatDog/validation'
weightsPath= "weights/"
modelPath= "model/"
weightsFilePath= "weights/weightscatsdogs.h5"
modelFilePath= "model/modelcatsdogs.json"
path = WORK_DIRECTORY+"UserData/"
nb_train_samples = 24158
nb_validation_samples = 1052
epochs =2
batch_size = 25
num_classes =2
input_shape = (image_width, image_height, 3)

In [ ]:
def HouseKeeping(pathName):
    pathName = WORK_DIRECTORY+ pathName
    if os.path.exists(pathName):
        shutil.rmtree(pathName)
    if not os.path.exists(pathName):
        os.makedirs(pathName)   

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=90.,
                                   zoom_range=0.3,
                                   shear_range=0.3)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model = Sequential()
 
#conv2d_1  32 filters of shape 3 X 3
#Parameters = 896 = ((Filter width * Filter Breadth * Channels) + 1) * Number of Filters
#((3 X 3 X 3 )+1) *32 ) = 896
model.add(Conv2D(32, (3, 3), input_shape=input_shape))     

# Filter Count * 4
# 32 * 4 = 128
model.add(BatchNormalization()) 
model.add(Activation('relu'))
#max_pooling2d_1
model.add(MaxPooling2D(pool_size=(2, 2)))   

#conv2d_2  64 filters of shape 3 X 3   
# (3 * 3 * 32 + 1) * 64 = 18496
model.add(Conv2D(64, (3, 3)))           

# Filter Count * 4
# 64 * 4 = 256
model.add(BatchNormalization())
model.add(Activation('relu'))
#max_pooling2d_2
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Dropout(0.5))
#conv2d_3  128 filters of shape 3 X 3    
# ( 3 * 3  * 64 +1)*128  = 73856
model.add(Conv2D(128, (3, 3)))        

# Filter Count * 4
# 128 * 4 = 512
model.add(BatchNormalization())
model.add(Activation('elu'))
#max_pooling2d_3
model.add(MaxPooling2D(pool_size=(2, 2)))  
 


model.add(Dropout(0.5))
model.add(Flatten())

# last Count = 512
# (last count + 1) * DenseCount
#(512 + 1)* 100 = 51300
model.add(Dense(100, activation='elu'))
 
# (100+1)*1 = 101
model.add(Dense(1,   activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
print(model.summary())


In [ ]:
HouseKeeping(weightsPath)
HouseKeeping(modelPath)
model_json = model.to_json()
with open(modelFilePath, "w") as json_file:
    json_file.write(model_json) 

In [ ]:
#+"{epoch:03d}-{val_acc:3f}.hdf5"
checkpointer = ModelCheckpoint(filepath=weightsFilePath,
                               verbose=1, 
                               monitor='val_acc',
                               save_best_only=True,
                               mode='max')

with tf.device('/gpu:0'):
    history=model.fit_generator(
        train_generator,
        verbose=1,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpointer])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss','val_loss'])
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(acc)
plt.plot(val_acc)
plt.legend(['acc','val_acc'])
plt.title("Accuracy")
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.show()

In [ ]:
files = os.listdir(path)

for file in files:
   
    t_image= image.load_img(path + file )
    print(path + file )
    test_image = image.load_img(path + file, target_size=(image_height, image_width))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    pred = model.predict_on_batch(test_image)
    print(pred)
    if pred >= 1.0:
        print("Dog")
    else:
        print("Cat")

    plt.imshow(t_image)
    plt.show()      